# 3. Parameter Definition for Optimization Models

This notebook is part of a step-by-step guide to creating an optimization model. We continue our 7-step procedure:

1. ✓ Import Pyomo and Define the Sets
2. ✓ Define the Decision Variables
3. ➡️ Define the Parameters
4. Define the Expressions
5. Define the Objective Function
6. Define the Constraints

In this notebook, we will explore how to define and use parameters in Pyomo. Parameters are the input data or coefficients in our optimization model. Unlike decision variables, parameters are known values that don't change during optimization.

We will cover:
- What are parameters and why are they important
- How to define scalar parameters
- How to define indexed parameters
- How to use parameters in expressions
- How to update parameter values
- Common patterns and best practices

## Setup: Importing Pyomo and Creating a Model

As in the previous notebooks, we start by importing Pyomo and creating a concrete model:

In [1]:
import pyomo.environ as pyo

# Create a concrete model
m = pyo.ConcreteModel()

Here, we initiate an empty concrete model `m`. We will add sets, variables, parameters into this model.

## What are Parameters?

Parameters are the constant coefficients in our optimization model. They represent known values that are used in:
- Objective function coefficients
- Constraint coefficients
- Bounds on variables
- Right-hand side values of constraints

For example, in a production planning problem:
- Unit prices
- Production costs
- Machine capacities
- Resource requirements
- Demand forecasts

are all parameters.

Let's use our previous example where we had:
- Products set $I$ and Machines set $J$
- Production quantities $x_i$ for each product $i \in I$
- Production time $y_{i,j}$ for product $i$ on machine $j$

We'll add parameters for:
- Unit profit for each product ($p_i$)
- Production cost per hour on each machine ($c_j$)
- Machine capacities in hours ($cap_j$)
- Processing time requirements for each product on each machine ($t_{i,j}$)

## Creating Sets and Variables

First, let's recreate our sets and variables from the previous notebooks:

In [2]:
# Create sets
m.I = pyo.Set(initialize=['P1', 'P2', 'P3'])  # Products
m.J = pyo.Set(initialize=['M1', 'M2'])       # Machines

# Production amount for each product
x_init = {'P1': 100, 'P2': 150, 'P3': 200}
m.x = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=x_init)

# Production time allocated to each product on each machine
y_init = {('P1', 'M1'): 2, ('P1', 'M2'): 3, ('P2', 'M1'): 3, ('P2', 'M2'): 2, ('P3', 'M1'): 4, ('P3', 'M2'): 3}
m.y = pyo.Var(m.I, m.J, domain=pyo.NonNegativeReals, initialize=y_init)

What we did here is to add set `I` and `J` to the model `m` and initialize the variables `x` and `y` with some values.  If these steps are not familiar to you, please refer to the previous notebooks in this series.

## Defining Parameters in Pyomo

### 1. Scalar Parameters

Scalar parameters are single values that don't need indexing. For example, let's define a minimum profit target:

In [3]:
# Define a scalar parameter
m.min_profit = pyo.Param(initialize=1000)

# Access the parameter value
m.min_profit.display()

min_profit : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
    Key  : Value
    None :  1000


### 2. Indexed Parameters

Indexed parameters are defined over sets. Let's define our production parameters:

In [4]:
# Unit profit for each product
profit_data = {'P1': 100, 'P2': 150, 'P3': 200}
m.p = pyo.Param(m.I, initialize=profit_data)

# Print parameters
m.p.display()

p : Size=3, Index=I, Domain=Any, Default=None, Mutable=False
    Key : Value
     P1 :   100
     P2 :   150
     P3 :   200


We can also access the parameter value using the set index:

In [5]:
print(m.p['P1'])


100


Similar as variables, we can iterate over the parameter values:

In [6]:
for i in m.I:
    print(i, m.p[i])

P1 100
P2 150
P3 200


Or, more compactly:

In [7]:
print([m.p[i] for i in m.I])

[100, 150, 200]


We can also use the summation operator:

In [8]:
#calculate the sum of the unit profits
sum_p = sum(m.p[i] for i in m.I)
print("sum_p = ", sum_p)

#calculate the total profit
total_profit = sum(m.p[i] * m.x[i] for i in m.I)
print("total_profit = ", total_profit)
print("total_profit value  = ", total_profit())

sum_p =  450
total_profit =  100*x[P1] + 150*x[P2] + 200*x[P3]
total_profit value  =  72500


Note here that we do not need to use `()` to access the value of Pyomo parameters. This is because Pyomo parameters are not functions, they are just values.

Meanwhile, we need to use `()` to access the value of Pyomo variables or expressions. 

Let's introduce a few more parameters:

In [9]:
# Production cost per hour on each machine
cost_data = {'M1': 50, 'M2': 40}
m.c = pyo.Param(m.J, initialize=cost_data)

# Machine capacities (hours available)
capacity_data = {'M1': 160, 'M2': 200}
m.cap = pyo.Param(m.J, initialize=capacity_data)

# Processing time requirements (hours per unit)
time_data = {
    ('P1','M1'): 2, ('P1','M2'): 3,
    ('P2','M1'): 3, ('P2','M2'): 2,
    ('P3','M1'): 4, ('P3','M2'): 3
}
m.t = pyo.Param(m.I, m.J, initialize=time_data)

Let's examine our parameters:

In [10]:
# Print unit profits
print("Unit Profits:")
for i in m.I:
    print(i, m.p[i])

# Print machine costs
print("\nMachine Costs:")
for j in m.J:
    print(j, m.c[j])

# Print machine capacities
print("\nMachine Capacities:")
for j in m.J:
    print(j, m.cap[j])

# Print processing times
print("\nProcessing Times:")
for i in m.I:
    for j in m.J:
        print(i, j, m.t[i,j])

Unit Profits:
P1 100
P2 150
P3 200

Machine Costs:
M1 50
M2 40

Machine Capacities:
M1 160
M2 200

Processing Times:
P1 M1 2
P1 M2 3
P2 M1 3
P2 M2 2
P3 M1 4
P3 M2 3


## Using Parameters in Expressions

The important role of parameters is to be used in expressions in our model. Let's create some common expressions:

In [11]:
# Total revenue expression
total_profit = sum(m.p[i] * m.x[i] for i in m.I)
print("Total profit expression: ", total_profit)
print("Total profit value: ", total_profit())

# Total production cost expression
total_cost = sum(m.c[j] * m.y[i,j] for i in m.I for j in m.J)
print("\nTotal cost expression: ", total_cost)
print("Total cost value: ", total_cost())

# Machine utilization for M1
m1_usage = sum(m.t[i,'M1'] * m.x[i] for i in m.I)
print("\nMachine M1 usage expression: ", m1_usage)
print("Machine M1 usage value: ", m1_usage())

Total profit expression:  100*x[P1] + 150*x[P2] + 200*x[P3]
Total profit value:  72500

Total cost expression:  50*y[P1,M1] + 40*y[P1,M2] + 50*y[P2,M1] + 40*y[P2,M2] + 50*y[P3,M1] + 40*y[P3,M2]
Total cost value:  770

Machine M1 usage expression:  2*x[P1] + 3*x[P2] + 4*x[P3]
Machine M1 usage value:  1450


## Mutable vs Immutable Parameters

By default, parameters in Pyomo are immutable (cannot be changed after initialization). However, we can create mutable parameters that can be updated by using `mutable=True` in the initialization. When we do this, we need to use `()` to access the value of the parameter, since the parameter will behave similar to a variable.

In [12]:
# Create a parameter for demand
demand_data = {'P1': 100, 'P2': 150, 'P3': 200}
m.d = pyo.Param(m.I, initialize=demand_data, mutable=True)

# Print original value
print("Original demand for P1: ", m.d['P1']())

# Update the parameter
m.d['P1'] = 120

# Print new value
print("Updated demand for P1: ", m.d['P1']())

Original demand for P1:  100
Updated demand for P1:  120


You will get an error if you try to update an immutable parameter, which is the default behavior in Pyomo. 

In most cases, we will use immutable parameters in our model in this course.


## Loading Parameters from External Sources

In practice, parameter values often come from external sources like CSV files or databases. Here's how we can load parameters from a dictionary (which could come from a CSV):

In [13]:
# Example data that might come from a CSV or database
quality_data = {
    'Product': ['P1', 'P2', 'P3'],
    'Quality_Rating': [95, 88, 92]
}

print(quality_data)

{'Product': ['P1', 'P2', 'P3'], 'Quality_Rating': [95, 88, 92]}


This is not a standard dictionary format for Pyomo. We need to convert it to a dictionary format for Pyomo, which needs the index to be the key and the value to be the value. We can use the `zip` function to do this.

In [14]:
# Convert to dictionary format for Pyomo
quality_dict = dict(zip(quality_data['Product'], quality_data['Quality_Rating']))

print(quality_dict)

{'P1': 95, 'P2': 88, 'P3': 92}


We can now use this dictionary to initialize our parameter.

In [15]:
# Create parameter
m.q = pyo.Param(m.I, initialize=quality_dict)

# Print values
print("Quality Ratings:")
for i in m.I:
    print(i, m.q[i])

Quality Ratings:
P1 95
P2 88
P3 92


## Default Values and Missing Data

We can specify default values for parameters when some data might be missing using the `default` keyword in the initialization. This is useful when we want to create a parameter with a default value for all indices that are not explicitly defined in the initialization.

In [16]:
# Setup times with some missing data
setup_data = {
    ('P1','M1'): 30, ('P1','M2'): 25,
    ('P3','M1'): 35, ('P3','M2'): 30
}

# Create parameter with default value
m.setup = pyo.Param(m.I, m.J, initialize=setup_data, default=0)

# Print values
print("Setup Times:")
for i in m.I:
    for j in m.J:
        print(i, j, m.setup[i,j])

Setup Times:
P1 M1 30
P1 M2 25
P2 M1 0
P2 M2 0
P3 M1 35
P3 M2 30


## Using Parameters in Common Production Planning Expressions

Let's see some typical expressions using our parameters:

In [17]:
# Net profit (revenue - cost)
net_profit = total_profit - total_cost
print("Net profit expression: ", net_profit)
print("Net profit value: ", net_profit())

# Quality-weighted production volume
quality_weighted = sum(m.q[i] * m.x[i] for i in m.I)
print("\nQuality-weighted production: ", quality_weighted)
print("Quality-weighted production value: ", quality_weighted())

# Total time including setup on M1
total_time_m1 = sum(m.setup[i,'M1'] + m.t[i,'M1'] * m.x[i] for i in m.I)
print("\nTotal setup and processing time on M1: ", total_time_m1)
print("Total setup and processing time on M1 value: ", total_time_m1())

Net profit expression:  100*x[P1] + 150*x[P2] + 200*x[P3] - (50*y[P1,M1] + 40*y[P1,M2] + 50*y[P2,M1] + 40*y[P2,M2] + 50*y[P3,M1] + 40*y[P3,M2])
Net profit value:  71730

Quality-weighted production:  95*x[P1] + 88*x[P2] + 92*x[P3]
Quality-weighted production value:  41100

Total setup and processing time on M1:  30 + 2*x[P1] + 3*x[P2] + 35 + 4*x[P3]
Total setup and processing time on M1 value:  1515


In the next notebook, we will see how to formalize these expressions and store them as expressions in our model, which will allow us to use them in constraints, objectives, and output metrics.

## Conclusion

We've covered the key aspects of parameters in Pyomo:

1. Parameter Types:
   - Scalar parameters (single values)
   - Indexed parameters (over sets)
   - Mutable vs immutable parameters

2. Parameter Definition:
   - Using initialize with values
   - Setting default values

3. Parameter Usage:
   - In mathematical expressions
   - Updating mutable parameters
   - Loading from external data

## Practice Exercises

Now it's your turn to practice working with parameters. Consider a situation where you're planning production across multiple time periods:

1. Create a model with:
   
   a. Products set $I = {P1, P2, P3}$

   b. Time periods set $T = {T1, T2, T3, T4}$
   
   c. Decision variables:
   
     - Production amount for each product in each time period $x_{i,t}$ with domain NonNegativeReals and initialize value 0
     - Storage amount for each product in each time period $y_{i,t}$ with domain NonNegativeReals and initialize value 0
   
2. Define the following parameters (immutable):
   a. Unit production cost for each product (scalar) $$c = 10$$
   b. Demand for each product in each time period (indexed) $$d = {P1: 100, P2: 150, P3: 200}$$
   c. Storage cost for each product (indexed) $$s = {P1: 10, P2: 15, P3: 20}$$
   d. Production capacity in each time period (indexed) $$cap = {T1: 160, T2: 200, T3: 180, T4: 220}$$
   
3. Create a mutable parameter for Unit price for each product (indexed) $$p = {P1: 20, P2: 30, P3: 40}$$

4. Print the expressions and their values:
   
   a. Total production cost: $$\sum_{i \in I} c \times x_{i,t}$$
   b. Total storage cost: $$\sum_{i \in I} \sum_{t \in T} s_i \times y_{i,t}$$
   c. Revenues at each time period: $$\sum_{i \in I} p_i \times x_{i,t}$$
   d. Total revenues: $$\sum_{i \in I} \sum_{t \in T} p_i \times x_{i,t}$$
   e. Net profit: $$\sum_{i \in I} \sum_{t \in T} p_i \times x_{i,t} - \sum_{i \in I} \sum_{t \in T} c \times x_{i,t} - \sum_{i \in I} \sum_{t \in T} s_i \times y_{i,t}$$

In [18]:
# 1. Create model, sets, variables
# Sets: I = {P1, P2, P3}, T = {T1, T2, T3, T4}
# Variables: x_{i,t}, y_{i,t}
### Write your code below ###


In [19]:
# 2. Define basic parameters
# c = 10, d = {P1: 100, P2: 150, P3: 200}, 
# s = {P1: 10, P2: 15, P3: 20}, 
# cap = {T1: 160, T2: 200, T3: 180, T4: 220}
### Write your code below ###


In [20]:
# 3. Create mutable price parameter
# p = {P1: 20, P2: 30, P3: 40}
### Write your code below ###


In [21]:
# 4. Print the expressions and their values:
# a. Total production cost: $$\sum_{i \in I} c \times x_{i,t}$$
### Write your code below ###


In [22]:
# 4. b. Total storage cost: $$\sum_{i \in I} \sum_{t \in T} s_i \times y_{i,t}$$
### Write your code below ###


In [23]:
# 4. c. Revenues at each time period: $$\sum_{i \in I} p_i \times x_{i,t}$$
### Write your code below ###


In [24]:
# 4. d. Total revenues: $$\sum_{i \in I} \sum_{t \in T} p_i \times x_{i,t}$$
### Write your code below ###


In [25]:
# 4. e. Net profit: $$\sum_{i \in I} \sum_{t \in T} p_i \times x_{i,t} - \sum_{i \in I} \sum_{t \in T} c \times x_{i,t} - \sum_{i \in I} \sum_{t \in T} s_i \times y_{i,t}$$
### Write your code below ###
